In [2]:
import eegPinelineDesign
import numpy as np
import random
import mne
import matplotlib.pyplot as plt
%matplotlib inline
import os
import pandas as pd
import re
import json
import scipy
from sklearn.linear_model import LogisticRegression
from sklearn.decomposition import PCA,FastICA
from sklearn.pipeline import Pipeline
from sklearn.grid_search import GridSearchCV
from sklearn.cross_validation import cross_val_score
from scipy.fftpack import fft,ifft
import math
from sklearn.metrics import classification_report,accuracy_score,confusion_matrix
from scipy.signal import spectrogram,find_peaks_cwt
from mne.preprocessing.ica import ICA
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import average_precision_score
from sklearn.cross_validation import train_test_split
from sklearn.preprocessing import label_binarize
from sklearn.multiclass import OneVsRestClassifier
from sklearn.svm import SVC
from sklearn.preprocessing import label_binarize
import itertools

In [3]:
eegPinelineDesign.change_file_directory('C:/Users/ning/Downloads/training set')
EDFfiles, Annotationfiles = eegPinelineDesign.split_type_of_files()

['64chlocs.elp', 'label_extraction.npy', 'label_extraction.pkl', 's5d2_final.edf', 's5d2_final_annotations.txt', 's6n2_final.edf', 's6n2_final_annotations.txt', 'Sleep Stage Scoring Criteria.docx', 'suj10_d1final.edf', 'suj10_d1final_annotations.txt', 'suj10_d2final.edf', 'suj10_d2final_annotations.txt', 'suj13_l2nap_day2 edited.edf', 'suj13_l2nap_day2 edited1.edf', 'suj13_l2nap_day2 edited1_annotations.txt', 'suj13_l2nap_day2 edited_annotations.txt', 'suj13_l2nap_day2 edited_C3.txt', 'suj13_l2nap_day2 edited_C4.txt', 'suj13_l2nap_day2 edited_F3.txt', 'suj13_l2nap_day2 edited_F4.txt', 'suj13_l2nap_day2 edited_O1.txt', 'suj13_l2nap_day2 edited_O2.txt', 'suj5_d1final.edf', 'suj5_d1final_annotations.txt', 'suj6_d1final.edf', 'suj6_d1final_annotations.txt', 'suj8_d1final.edf', 'suj8_d1final_annotations.txt', 'suj8_d2final.edf', 'suj8_d2final_annotations.txt', 'suj9_d1final.edf', 'suj9_d1final_annotations.txt', 'suj9_d2final.edf', 'suj9_d2final_annotations.txt', 'testing-montage-2.mtg', 'Tr

In [6]:
channelList = ['F3','F4','C3','C4','O1','O2','ROC','LOC']
file_to_read, fileName = eegPinelineDesign.pick_sample_file(EDFfiles,n=2)
raw = eegPinelineDesign.load_data(file_to_read,channelList,None, 499)
annotation_to_read = [x for x in Annotationfiles if fileName in x]
file = pd.read_csv(annotation_to_read[0])

Extracting edf Parameters from suj10_d1final.edf...
Setting channel info structure...
Creating Raw.info structure...
Reading 0 ... 3601999  =      0.000 ...  3601.999 secs...
[done]
Ready.
Fitting ICA to data using 8 channels. 
Please be patient, this may take some time
Inferring max_pca_components from picks.
Using all PCA components: 8
    Searching for artifacts...
    found 2 artifacts by EOG 00
    found 1 artifact by skewness
    found 1 artifact by kurtosis
    found 1 artifact by variance
Artifact indices found:
    2, 7, 4, 4, 2
    Removing duplicate indices...
Ready.
Transforming to ICA space (8 components)
Zeroing out 3 ICA components
Inverse transforming to PCA space
Reconstructing sensor space signals from 8 PCA components
Low-pass filtering at 5e+02 Hz
Low-pass filtering at 5e+02 Hz


In [9]:
labelFind = re.compile('spindle',re.IGNORECASE)
spindles=[]
for row in file.iterrows():
    currentEvent = row[1][-1]
    if labelFind.search(currentEvent):
        spindles.append(row[1][0])

In [11]:
ratio_index={}
muscle_index={}
sample_ind={}
for ii,names in enumerate(channelList):
    if names == 'ROC' or names == 'LOC':
        pass
    else:
        windowsize=2;ratio_index[names]=[];muscle_index[names]=[];sample_ind[names]=[]
        endPoint = 2;startPoint=endPoint-windowsize
        distance_to_end = raw.last_samp/1000 - endPoint
        while distance_to_end >0:
            start,stop = raw.time_as_index([startPoint,endPoint])
            segment, time = raw[ii,start:stop];
            filter_alpha = mne.filter.band_pass_filter(segment,1000,8,12);
            RMS_alpha = np.sqrt(sum(filter_alpha[0]**2)/len(filter_alpha))
            filter_spindle = mne.filter.band_pass_filter(segment,1000,11.5,16)
            RMS_spindle = np.sqrt(sum(filter_spindle[0]**2))/len(filter_spindle)
            filter_muscle = mne.filter.band_pass_filter(segment,1000,30,40)
            RMS_muscle = np.sqrt(sum(filter_muscle[0]**2)/len(filter_muscle))
            #print(RMS_alpha,RMS_spindle,RMS_muscle)
            point = 0
            if RMS_alpha/RMS_spindle < 1.2:
                point +=1
                ratio_index[names].append(1e-1)
            else:
                ratio_index[names].append(-1e-1)
            if RMS_muscle >5*10e-4:
                point -=1
                muscle_index[names].append(-1e-1)
            else:
                muscle_index[names].append(-1e-1)
            if point ==1:
                sample_ind[names].append(1e-1)
            elif point ==0:
                sample_ind[names].append(1/6*1e-1)
            else:
                sample_ind[names].append(-1e-1)
            
            
            endPoint += windowsize/2#determine step size
            startPoint=endPoint-windowsize
            distance_to_end = raw.last_samp/1000 - endPoint 

c:\anaconda3\lib\site-packages\mne\filter.py:333: UserWarning: Attenuation at stop frequency 7.5Hz is only 9.0dB.
  '%0.1fdB.' % (att_freq, att_db))
c:\anaconda3\lib\site-packages\mne\filter.py:333: UserWarning: Attenuation at stop frequency 16.5Hz is only 8.9dB.
  '%0.1fdB.' % (att_freq, att_db))
c:\anaconda3\lib\site-packages\mne\filter.py:333: UserWarning: Attenuation at stop frequency 29.5Hz is only 9.1dB.
  '%0.1fdB.' % (att_freq, att_db))
